In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\dylan\AppData\Local\Temp\ipykernel_31860\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [ ]:
# ! pip install nltk 

In [55]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dylan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [128]:
import pandas as pd
import re
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from collections import Counter

from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

import nltk

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [96]:
## Read Data for the Fraudulent Email Kaggle Challenge
data_train = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data_train = data_train.head(1000)
print(data_train.shape)
data_train.fillna("",inplace=True)

(1000, 2)


In [98]:
data_train.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


### Let's divide the training and test set into two partitions

In [101]:
# Your code
train, test = train_test_split(data, test_size=0.2, random_state=42)

print(f"Training set size: {len(train)}")
print(f"Test set size: {len(test)}")

Training set size: 800
Test set size: 200


## Data Preprocessing

In [104]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [107]:
# Your code
# Function to clean HTML
def clean_html(html):
    # Step 1: Remove inline JavaScript/CSS
    no_js_css = re.sub(r'<(script|style)[^>]*>.*?</\1>', '', html, flags=re.DOTALL)
    # Step 2: Remove HTML comments
    no_comments = re.sub(r'<!--.*?-->', '', no_js_css, flags=re.DOTALL)
    # Step 3: Remove remaining HTML tags
    clean_text = re.sub(r'<[^>]+>', '', no_comments)
    # Step 4: Strip excess whitespace
    return clean_text.strip()

# Apply the cleaning function to the DataFrame column
data_train['cleaned_column'] = data_train['text'].apply(clean_html)

# Display the cleaned DataFrame
print(data_train.head())

                                                text  label  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1   
1                                           Will do.      0   
2  Nora--Cheryl has emailed dozens of memos about...      0   
3  Dear Sir=2FMadam=2C I know that this proposal ...      1   
4                                                fyi      0   

                                      cleaned_column  
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...  
1                                           Will do.  
2  Nora--Cheryl has emailed dozens of memos about...  
3  Dear Sir=2FMadam=2C I know that this proposal ...  
4                                                fyi  


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [110]:
# Your code
# Function to clean HTML and process text
def clean_html_and_text(html):
    # Step 1: Remove inline JavaScript/CSS
    no_js_css = re.sub(r'<(script|style)[^>]*>.*?</\1>', '', html, flags=re.DOTALL)
    # Step 2: Remove HTML comments
    no_comments = re.sub(r'<!--.*?-->', '', no_js_css, flags=re.DOTALL)
    # Step 3: Remove remaining HTML tags
    text = re.sub(r'<[^>]+>', '', no_comments)
    # Step 4: Strip excess whitespace
    text = text.strip()
    
    # Additional cleaning steps:
    # Remove all special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove all single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    # Remove single characters from the start
    text = re.sub(r'^[a-zA-Z]\s+', '', text)
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Remove prefixed 'b' (if present, for bytes-like objects)
    text = re.sub(r'^b\s+', '', text)
    # Convert to lowercase
    text = text.lower()
    
    return text

# Apply the cleaning function to the DataFrame column
data_train['cleaned_column'] = data_train['text'].apply(clean_html_and_text)

# Display the cleaned DataFrame
print(data_train.head())

                                                text  label  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1   
1                                           Will do.      0   
2  Nora--Cheryl has emailed dozens of memos about...      0   
3  Dear Sir=2FMadam=2C I know that this proposal ...      1   
4                                                fyi      0   

                                      cleaned_column  
0  dear sir strictly private business proposal am...  
1                                            will do  
2  noracheryl has emailed dozens of memos about h...  
3  dear sirfmadamc know that this proposal might ...  
4                                                fyi  


## Now let's work on removing stopwords
Remove the stopwords.

In [113]:
# Your code
# List of stopwords to remove
stopwords = {'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each'}

# Function to clean HTML and process text
def clean_html_and_text(html):
    # Step 1: Remove inline JavaScript/CSS
    no_js_css = re.sub(r'<(script|style)[^>]*>.*?</\1>', '', html, flags=re.DOTALL)
    # Step 2: Remove HTML comments
    no_comments = re.sub(r'<!--.*?-->', '', no_js_css, flags=re.DOTALL)
    # Step 3: Remove remaining HTML tags
    text = re.sub(r'<[^>]+>', '', no_comments)
    # Step 4: Strip excess whitespace
    text = text.strip()
    
    # Additional cleaning steps:
    # Remove all special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove all single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    # Remove single characters from the start
    text = re.sub(r'^[a-zA-Z]\s+', '', text)
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Remove prefixed 'b'
    text = re.sub(r'^b\s+', '', text)
    # Convert to lowercase
    text = text.lower()
    
    # Remove stopwords
    words = text.split()
    words = [word for word in words if word not in stopwords]
    text = ' '.join(words)
    
    return text

# Apply the cleaning function to the DataFrame column
data_train['cleaned_column'] = data_train['text'].apply(clean_html_and_text)

# Display the cleaned DataFrame
print(data_train.head())

                                                text  label  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1   
1                                           Will do.      0   
2  Nora--Cheryl has emailed dozens of memos about...      0   
3  Dear Sir=2FMadam=2C I know that this proposal ...      1   
4                                                fyi      0   

                                      cleaned_column  
0  dear sir strictly private business proposal am...  
1                                            will do  
2  noracheryl has emailed dozens of memos about h...  
3  dear sirfmadamc know that this proposal might ...  
4                                                fyi  


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [116]:
# Ensure NLTK resources are downloaded
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dylan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dylan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dylan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dylan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [118]:
# Your code
# List of stopwords to remove
stopwords = {'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each'}

# Initialize WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Function to clean HTML and process text
def clean_html_and_text(html):
    # Step 1: Remove inline JavaScript/CSS
    no_js_css = re.sub(r'<(script|style)[^>]*>.*?</\1>', '', html, flags=re.DOTALL)
    # Step 2: Remove HTML comments
    no_comments = re.sub(r'<!--.*?-->', '', no_js_css, flags=re.DOTALL)
    # Step 3: Remove remaining HTML tags
    text = re.sub(r'<[^>]+>', '', no_comments)
    # Step 4: Strip excess whitespace
    text = text.strip()
    
    # Additional cleaning steps:
    # Remove all special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove all single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    # Remove single characters from the start
    text = re.sub(r'^[a-zA-Z]\s+', '', text)
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Remove prefixed 'b'
    text = re.sub(r'^b\s+', '', text)
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize text into words
    words = word_tokenize(text)
    
    # Remove stopwords
    words = [word for word in words if word not in stopwords]
    
    # Lemmatize each word
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    
    # Return the lemmatized words as a list
    return lemmatized_words

# Apply the cleaning function to the DataFrame column
data_train['cleaned_column'] = data_train['text'].apply(clean_html_and_text)

# Display the cleaned DataFrame
print(data_train.head())

                                                text  label  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1   
1                                           Will do.      0   
2  Nora--Cheryl has emailed dozens of memos about...      0   
3  Dear Sir=2FMadam=2C I know that this proposal ...      1   
4                                                fyi      0   

                                      cleaned_column  
0  [dear, sir, strictly, private, business, propo...  
1                                         [will, do]  
2  [noracheryl, ha, emailed, dozen, of, memo, abo...  
3  [dear, sirfmadamc, know, that, this, proposal,...  
4                                              [fyi]  


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [121]:
# Your code
# Stopwords to remove
stopwords = {'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'you', 'your', 'me', 'are'}

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to clean and preprocess text
def clean_and_tokenize(text):
    # Remove special characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    # Tokenize
    words = word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word not in stopwords]
    # Lemmatize words
    words = [lemmatizer.lemmatize(word) for word in words]
    return words

# Apply cleaning and tokenization
data_train['cleaned_message'] = data_train['text'].apply(clean_and_tokenize)

# Separate ham and spam messages
ham_words = [word for message in data_train[data_train['label'] == 0]['cleaned_message'] for word in message]
spam_words = [word for message in data_train[data_train['label'] == 1]['cleaned_message'] for word in message]

# Count word frequencies
ham_word_counts = Counter(ham_words)
spam_word_counts = Counter(spam_words)

# Get the top 10 words for ham and spam
top_ham_words = ham_word_counts.most_common(10)
top_spam_words = spam_word_counts.most_common(10)

# Display the results
print("Top 10 words in ham messages:")
for word, count in top_ham_words:
    print(f"{word}: {count}")

print("\nTop 10 words in spam messages:")
for word, count in top_spam_words:
    print(f"{word}: {count}")

Top 10 words in ham messages:
the: 1725
to: 1058
and: 819
a: 816
of: 792
in: 589
that: 393
is: 382
for: 367
it: 319

Top 10 words in spam messages:
the: 6847
to: 5528
of: 4909
a: 3991
and: 3918
in: 3202
i: 2896
this: 2573
my: 2047
for: 1994


## Extra features

In [136]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['cleaned_column'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['cleaned_column'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['cleaned_column'].apply(lambda x: len(x)) 

'''data_val['money_mark'] = data_val['cleaned_column'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['cleaned_column'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['cleaned_column'].apply(lambda x: len(x)) 
'''
data_train.head()

,text,label,cleaned_column,cleaned_message,money_mark,suspicious_words,text_len
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,"[dear, sir, strictly, private, business, propo...","[dear, sir, strictly, a, private, business, pr...",NaN,NaN,374
1,Will do.,0,"[will, do]","[will, do]",NaN,NaN,2
2,Nora--Cheryl has emailed dozens of memos about...,0,"[noracheryl, ha, emailed, dozen, of, memo, abo...","[noracheryl, ha, emailed, dozen, of, memo, abo...",NaN,NaN,37
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,"[dear, sirfmadamc, know, that, this, proposal,...","[dear, sirfmadamc, i, know, that, this, propos...",NaN,NaN,330
4,fyi,0,[fyi],[fyi],NaN,NaN,1


## How would work the Bag of Words with Count Vectorizer concept?

In [142]:
# Your code
# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the text data
bow_matrix = vectorizer.fit_transform(data_train['cleaned_column'])

# Get feature names (vocabulary)
vocabulary = vectorizer.get_feature_names_out()

# Convert the sparse matrix to a dense DataFrame for better visualization
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vocabulary)

print("Vocabulary:", vocabulary)
print("\nBag of Words Matrix:")
print(bow_df)

Vocabulary: ['aac' 'aaclocated' 'aae' ... 'zumadirector' 'zumae' 'zurich']

Bag of Words Matrix:
     aac  aaclocated  aae  aag  aaronovitchon  abacha  abachabefore  abachac  \
0      0           0    0    0              0       0             0        0   
1      0           0    0    0              0       0             0        0   
2      0           0    0    0              0       0             0        0   
3      0           0    0    0              0       0             0        0   
4      0           0    0    0              0       0             0        0   
..   ...         ...  ...  ...            ...     ...           ...      ...   
995    0           0    0    0              0       0             0        0   
996    0           0    0    0              0       0             0        0   
997    0           0    0    0              0       0             0        0   
998    0           0    0    0              0       0             0        0   
999    0           0   

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [140]:
# Your code
# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# If the 'message' column contains lists of tokens, join them into strings
if isinstance(data_train['cleaned_column'][0], list):
    data_train['cleaned_column'] = data_train['cleaned_column'].apply(lambda tokens: ' '.join(tokens))
    
# Fit and transform the dataset
tfidf_matrix = tfidf_vectorizer.fit_transform(data_train['cleaned_column'])

# Print the shape of the vectorized dataset
print(f"Shape of the TF-IDF vectorized dataset: {tfidf_matrix.shape}")

# Display the feature names (vocabulary)
print("\nFeature Names (Vocabulary):")
print(tfidf_vectorizer.get_feature_names_out())

# Convert the sparse TF-IDF matrix to a dense DataFrame for better visualization
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
print("\nTF-IDF Matrix:")
print(tfidf_df)

Shape of the TF-IDF vectorized dataset: (1000, 19454)

Feature Names (Vocabulary):
['aac' 'aaclocated' 'aae' ... 'zumadirector' 'zumae' 'zurich']

TF-IDF Matrix:
     aac  aaclocated  aae  aag  aaronovitchon  abacha  abachabefore  abachac  \
0    0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
1    0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
2    0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
3    0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
4    0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
..   ...         ...  ...  ...            ...     ...           ...      ...   
995  0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
996  0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
997  0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
998  0.0         0.0  0.0  0.0        

## And the Train a Classifier?

In [144]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [146]:
# Your code
# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Vectorize the dataset
X = tfidf_vectorizer.fit_transform(data_train['cleaned_column'])

# Target variable (labels)
y = data_train['label']

In [148]:
# Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the size of train/test sets
print(f"Train set size: {X_train.shape[0]} documents")
print(f"Test set size: {X_test.shape[0]} documents")

Train set size: 800 documents
Test set size: 200 documents


In [150]:
# Initialize the Naive Bayes classifier
classifier = MultinomialNB()

# Train the classifier
classifier.fit(X_train, y_train)

MultinomialNB()

In [152]:
# Make predictions
y_pred = classifier.predict(X_test)

# Evaluate the classifier performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8800

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.82      0.89       125
           1       0.76      0.99      0.86        75

    accuracy                           0.88       200
   macro avg       0.88      0.90      0.88       200
weighted avg       0.91      0.88      0.88       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code